In [83]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, text, select
import urllib

# Step 1: Setup connection
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LAPTOP-ECJLSAST\\SQLExpress;"
    "DATABASE=master;"  # change to 'BankDB' if you created it
    "Trusted_Connection=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Step 2: Metadata setup
metadata = MetaData()

# Step 3: Define the accounts table
accounts_table = Table(
    'accounts',
    metadata,
    Column('account_id', Integer, primary_key=True),
    Column('account_holder', String(100), nullable=False),
    Column('account_type', String(50), nullable=False),
    Column('balance', Integer, nullable=False)
)



In [84]:
# Insert new records into accounts table
try:
    with engine.begin() as conn:  # ✅ use begin() to auto-commit
        conn.execute(
            accounts_table.insert(),
            [
                {'account_id': 103, 'account_holder': 'Ajay', 'account_type': 'Savings', 'balance': 8000},
                {'account_id': 104, 'account_holder': 'Sita', 'account_type': 'Current', 'balance': 15000}
            ]
        )
        print(" New accounts inserted.")
except Exception as e:
    print(" Insertion failed:", e)


 Insertion failed: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__accounts__46A222CD013DC830'. Cannot insert duplicate key in object 'dbo.accounts'. The duplicate key value is (103). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")
[SQL: INSERT INTO accounts (account_id, account_holder, account_type, balance) VALUES (?, ?, ?, ?), (?, ?, ?, ?)]
[parameters: (103, 'Ajay', 'Savings', 8000, 104, 'Sita', 'Current', 15000)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [85]:
# Read and display all account records
try:
    with engine.connect() as conn:
        result = conn.execute(select(accounts_table))
        print(" Account Table Data:")
        for row in result:
            print(row)
except Exception as e:
    print(" Reading accounts failed:", e)


 Account Table Data:
(101, 'Yasaswini', 'Savings', 5000)
(102, 'Krishna', 'Current', 12000)
(103, 'Ajay', 'Savings', 8000)
(104, 'Sita', 'Current', 15000)


In [86]:
# Re-read to confirm the update
try:
    with engine.connect() as conn:
        result = conn.execute(select(accounts_table))
        print(" Updated Account Table:")
        for row in result:
            print(row)
except Exception as e:
    print(" Reading updated data failed:", e)


 Updated Account Table:
(101, 'Yasaswini', 'Savings', 5000)
(102, 'Krishna', 'Current', 12000)
(103, 'Ajay', 'Savings', 8000)
(104, 'Sita', 'Current', 15000)


In [87]:
from sqlalchemy import update

try:
    with engine.connect() as conn:
        # Add 2000 to balance of account_id = 101
        conn.execute(
            update(accounts_table)
            .where(accounts_table.c.account_id == 101)
            .values(balance=accounts_table.c.balance + 2000)
        )
        print(" Balance updated successfully.")
except Exception as e:
    print(" Balance update failed:", e)


 Balance updated successfully.


In [88]:
stmt = select(accounts_table.c.account_id).where(accounts_table.c.account_id == 101)
with engine.begin() as conn:
    result = conn.execute(stmt).fetchone()
    if result:
        print(" Account already exists.")
    else:
        conn.execute(
            accounts_table.insert().values(account_id=101, account_holder='Yasaswini', account_type='Savings', balance=5000)
        )
        print(" Inserted new account.")


 Account already exists.


In [89]:

try:
    with engine.begin() as conn:
        result = conn.execute(select(accounts_table))
        print("Account Table Data:")
        rows = result.fetchall()
        if rows:
            for row in rows:
                print(row)
        else:
            print("⚠️ No records found.")
except Exception as e:
    print(" Error fetching data:", e)


Account Table Data:
(101, 'Yasaswini', 'Savings', 5000)
(102, 'Krishna', 'Current', 12000)
(103, 'Ajay', 'Savings', 8000)
(104, 'Sita', 'Current', 15000)
